In [1]:
import pandas as pd 
import sqlite3
import matplotlib.pyplot as plt

In [111]:
db = pd.read_csv("Sample-Superstore.csv",encoding='latin1' )

In [120]:
rename_dict = {
        'Row ID': 'RowID',
        'Order ID': 'OrderID',
        'Order Date': 'OrderDate',
        'Ship Date': 'ShipDate',
        'Ship Mode': 'ShipMode',
        'Customer ID': 'CustomerID',
        'Customer Name': 'CustomerName',
        'Postal Code': 'PostalCode',
        'Product ID': 'ProductID',
        'Sub-Category': 'SubCategory',
        'Product Name': 'ProductName',
        'Catalogue price': 'CataloguePrice',
        'Supplier price': 'SupplierPrice'
        }

# call rename () method
db.rename(columns=rename_dict,
          inplace=True)

In [133]:
db[['OrderDate','ShipDate']] = db[['OrderDate','ShipDate']].apply(pd.to_datetime)
db.head()

,RowID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,...,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit,Price,DiscountedPrice,PurchasePrice
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,130.98,130.9800,110.0232
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,243.98,243.9800,170.7860
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,7.31,7.3100,3.8743
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,348.21,191.5155,268.1217
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,13.98,11.1840,9.9258


In [132]:
db["Price"] = (db.Sales/db.Quantity)/(1-db.Discount)


db["DiscountedPrice"] = (db.Sales/db.Quantity)
db["PurchasePrice"] = (db.Sales-db.Profit)/db.Quantity

In [159]:
#Create rables dataset
Customers = db[["CustomerID", "CustomerName", "Segment"]]
Address = db[["CustomerID","Country" ,"Region", "State", "City", "PostalCode"]] # Address ID will be required to connect this to orders. Removed country as not required
Products = db[["ProductID", "ProductName","Category","SubCategory"]] 
ProductPrice = db[["ProductID","PurchasePrice", "Price", "DiscountedPrice"]]
Orders = db[["OrderID", "CustomerID", "OrderDate", "ShipDate", "ShipMode"]] #Needs Address ID!
Sales = db[["OrderID", "ProductID", "Quantity", "Sales", "Discount", "Profit"]] #Sales table did not survive 3NF

In [160]:
Customers["CustomerID"].unique().size
Customers["CustomerName"].unique().size
Customers = Customers.drop_duplicates(ignore_index=True)
Customers

,CustomerID,CustomerName,Segment
0,CG-12520,Claire Gute,Consumer
1,DV-13045,Darrin Van Huff,Corporate
2,SO-20335,Sean O'Donnell,Consumer
3,BH-11710,Brosina Hoffman,Consumer
4,AA-10480,Andrew Allen,Consumer
...,...,...,...
788,CJ-11875,Carl Jackson,Corporate
789,RS-19870,Roy Skaria,Home Office
790,SC-20845,Sung Chung,Consumer
791,RE-19405,Ricardo Emerson,Consumer


In [144]:
Address.isnull().sum()

CustomerID    0
Region        0
State         0
City          0
PostalCode    0
dtype: int64

In [165]:
#create AdressID
#Address.CustomerID.nunique()
Address.groupby(by=["CustomerID", "PostalCode"]).ngroups
Address["PostID"]=Address.groupby(['CustomerID','Country', 'City', 'State', 'PostalCode', 'Region']).ngroup() +1
Address["PostID"] = "A" + Address['PostID'].astype(str).str.zfill(5)
Address=Address.drop_duplicates()
Address=Address.reset_index(drop=True)
Address


,CustomerID,Country,Region,State,City,PostalCode,PostID
0,CG-12520,United States,South,Kentucky,Henderson,42420,A00860
1,DV-13045,United States,West,California,Los Angeles,90036,A01437
2,SO-20335,United States,South,Florida,Fort Lauderdale,33311,A04367
3,BH-11710,United States,West,California,Los Angeles,90032,A00539
4,AA-10480,United States,South,North Carolina,Concord,28027,A00015
...,...,...,...,...,...,...,...
4905,ML-17410,United States,West,California,Los Angeles,90008,A03206
4906,RA-19885,United States,South,Georgia,Athens,30605,A03806
4907,TB-21400,United States,South,Florida,Miami,33180,A04568
4908,DB-13060,United States,West,California,Costa Mesa,92627,A01134


In [196]:
Products = Products.drop_duplicates(subset=["ProductID", "ProductName"])

In [197]:
duplicated_products=Products.groupby("ProductID")["ProductName"].nunique()
duplicated_products = duplicated_products[duplicated_products > 1].index
Products['NewProductID'] = Products['ProductID']
# Iterate through the duplicated ProductIDs and assign new IDs for each unique ProductName
for product_id in duplicated_products:
    # Filter rows with this ProductID
    product_group = Products[Products['ProductID'] == product_id]
    
    # Enumerate through the rows with this ProductID and assign new unique ProductIDs
    for i, idx in enumerate(product_group.index):
        # Create a new ProductID by appending a suffix or a numeric sequence
        Products.at[idx, 'NewProductID'] = f"{product_id}-{i+1}"

# Step 3: Inspect the result
# Display the DataFrame with old and new ProductIDs
Products[['ProductID', 'ProductName', 'NewProductID']].sort_values(by="ProductID",ascending=True).head(30)




,ProductID,ProductName,NewProductID
3512,FUR-BO-10000112,"Bush Birmingham Collection Bookcase, Dark Cherry",FUR-BO-10000112
1760,FUR-BO-10000330,"Sauder Camden County Barrister Bookcase, Plank...",FUR-BO-10000330
1594,FUR-BO-10000362,Sauder Inglewood Library Bookcases,FUR-BO-10000362
1932,FUR-BO-10000468,O'Sullivan 2-Shelf Heavy-Duty Bookcases,FUR-BO-10000468
4088,FUR-BO-10000711,"Hon Metal Bookcases, Gray",FUR-BO-10000711
2403,FUR-BO-10000780,O'Sullivan Plantations 2-Door Library in Landv...,FUR-BO-10000780
485,FUR-BO-10001337,O'Sullivan Living Dimensions 2-Shelf Bookcases,FUR-BO-10001337
841,FUR-BO-10001519,O'Sullivan 3-Shelf Heavy-Duty Bookcases,FUR-BO-10001519
7967,FUR-BO-10001567,"Bush Westfield Collection Bookcases, Dark Cher...",FUR-BO-10001567
472,FUR-BO-10001601,"Sauder Mission Library with Doors, Fruitwood F...",FUR-BO-10001601


In [209]:
#Products.drop("ProductID", axis=1, inplace=True)
Products.rename(columns={"NewProductID":"ProductID"}, inplace=True)
Products.index


Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       9214, 9319, 9378, 9455, 9456, 9521, 9562, 9604, 9673, 9935],
      dtype='int64', length=1894)

In [214]:
#Change also the ProductID of other tables 
#ProductPrice.iloc[Products.index,[0]] = Products["ProductID"]
#Sales.iloc[Products.index,[0]] = Products["ProductID"]


In [220]:
#ProductPrice.drop_duplicates(subset="ProductID", inplace=True)
ProductPrice



,ProductID,PurchasePrice,Price,DiscountedPrice
0,FUR-BO-10001798,110.0232,130.98,130.9800
1,FUR-CH-10000454,170.7860,243.98,243.9800
2,OFF-LA-10000240,3.8743,7.31,7.3100
3,FUR-TA-10000577,268.1217,348.21,191.5155
4,OFF-ST-10000760,9.9258,13.98,11.1840
...,...,...,...,...
9521,TEC-PH-10002817,95.8429,134.99,107.9920
9562,TEC-MA-10003589,129.9636,249.93,74.9790
9604,OFF-AP-10003099,35.5896,49.43,49.4300
9673,TEC-PH-10002645,374.9925,499.99,299.9940
